In [4]:
# This file is auto-generated by [FlowGen](https://github.com/tiwater/flowgen)
# Last generated: 2024-03-02 14:07:20
#
# Autoflow Name: Search and Write
# Description: 
"""
Search related news on Bing and write article based on the news.
"""
#

"""
Notes:

Search related news on Bing and write article based on the news.

Question:
```md
search the lastest news about Elon Musk, and generate an article on it, a very detailed article.
```
"""


from dotenv import load_dotenv
load_dotenv()  # This will load all environment variables from .env

import argparse
import os
import time
from termcolor import colored

# Parse command line arguments
parser = argparse.ArgumentParser(description='Start a chat with agents.')
parser.add_argument('message', type=str, help='The message to send to agent.')
args = parser.parse_args()

import autogen

# openai, whisper are optional dependencies
# However, we beleive they are useful for other future examples, so we include them here as part of standard imports
from openai import OpenAI

from IPython import get_ipython
from autogen import AssistantAgent
from autogen import UserProxyAgent
from autogen import GroupChat
from autogen import GroupChatManager

# Replace the default get_human_input function for status control
def custom_get_human_input(self, prompt: str) -> str:
    # Set wait_for_human_input to True
    print('__STATUS_WAIT_FOR_HUMAN_INPUT__', prompt, flush=True)
    reply = input(prompt)
    # Restore the status to running
    print('__STATUS_RECEIVED_HUMAN_INPUT__', prompt, flush=True)
    return reply

autogen.ConversableAgent.get_human_input = custom_get_human_input


config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ['ollama'],
    },
)

llm_config = {
    "functions": [
        {
            "name": "search_bing_news",
            "description": "Search Bing for the question and return a list of found links.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "The search keyword for Bing"
                    }
                },
                "required": [
                  "keyword" 
                ]
            }
        }
    ],
    "config_list": config_list,
    "temperature": 0.5,
    "max_tokens": 1024,
}
group_llm_config={
    "config_list": config_list,
    "temperature": 0.5,
    "max_tokens": 1024,
}

node_9kl7tty5ght = AssistantAgent(
    name="Searcher",
    system_message="""Searcher. You response for search content, using certain tools get information you need.""",
    llm_config=llm_config,
)

node_s0758kaiwb = AssistantAgent(
    name="Writer",
    system_message="""Writer: you response for write article based on information you have. When finish writing, please append TERMINATE as newline to stop the conversation.""",
    llm_config=llm_config,
)


user_proxy = UserProxyAgent(
    name="UserProxy",
    system_message="When a link is provided, you should ask the assistant for fetching the content. Reply TERMINATE if the task has been solved at full satisfaction.Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=10,
    code_execution_config={
      "work_dir": "work_dir",
    },
)
# Function template content generator

def search_bing_news(keyword):
    import requests
    from bs4 import BeautifulSoup
    
    # Construct the search URL
    search_url = f"""https://www.bing.com/news/search?q={keyword.replace(' ', '+')}&form=QBN&pq={keyword.replace(' ', '+')}&sc=8-0&sp=-1&qs=n&sk="""
    
    # Perform the HTTP request to Bing
    response = requests.get(search_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all news articles
        articles = soup.find_all('div', {'class': 'news-card newsitem cardcommon'})
        # Extract the links from the news articles
        links = [article.find('a', href=True)['href'] for article in articles]
        return links
    else:
        return []
# register the functions
user_proxy.register_function(
    function_map={
      "search_bing_news": search_bing_news,
    }
)
# group chat
groupchat_2xle9ojpxwy = GroupChat(agents=[
        node_9kl7tty5ght,
        node_s0758kaiwb,
        user_proxy,
    ],
    messages=[],
)

node_2xle9ojpxwy = GroupChatManager(groupchat=groupchat_2xle9ojpxwy, llm_config=group_llm_config)

# message-"search the lastest news about Elon Musk, and generate an article on it, a very detailed article."


# Start the conversation
user_proxy.initiate_chat(
    node_2xle9ojpxwy,
    message=args.message,
    )

usage: ipykernel_launcher.py [-h] message
ipykernel_launcher.py: error: the following arguments are required: message


SystemExit: 2